In [1]:
import os

from IPython.display import display, Markdown, Image

In [2]:
REPO_PATH = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
TASK_PATH = os.path.join(REPO_PATH, "tasks", "09-textual-entailment.md")
DATA_PATH = '/home/dima/Projects/snli_1.0'

In [3]:
def show_markdown(path):
    with open(path, 'r') as fh:
        content = fh.read()
    display(Markdown(content))

In [4]:
show_markdown(TASK_PATH)

# Логічне слідування

## Завдання

Розробіть класифікатор, який приймає на вхід текст та гіпотезу і визначає зв'язок між ними за трьома класами:
- entailment (гіпотеза логічно слідує з тексту)
- contradiction (гіпотеза суперечить тексту)
- neutral (гіпотеза і текст не пов'язані)

Побудуйте базове рішення та ітеративно покращуйте його, додаючи ознаки. Обов'язково випробуйте ознаки лексичної, граматичної та семантичної схожості:
* До ознак *лексичної схожості* належить частка сутностей, слів, енграмів, іменників, дієслів, числівників тощо, які перетинаються в тексті та гіпотезі. Спробуйте лематизацію чи стемінг, опрацюйте заперечення, нормалізуйте дані.
* До ознак *граматичної схожості* належить частка синтаксичних структур чи залежностей, які перетинаються в тексті та гіпотезі. Спробуйте або дерева складників, або дерева залежностей, або і те, і друге.
* До ознак *семантичної схожості* належить:
  1. Наявність лексико-семантичних зв'язків між словами в тексті та в гіпотезі. Спробуйте виявити наявність синонімів, антонімів, гіперонімів, гіпонімів, пов'язаних слів, логічного слідування тощо. Ви можете використати будь-яку онтологію ([WordNet](https://wordnet.princeton.edu/), [ConceptNet](http://conceptnet.io/), [BabelNet](https://babelnet.org/) тощо) та будь-яку бібліотеку для роботи з нею.
  2. *[Опційно]* Схожість семантичних ролей в тексті та гіпотезі. Спробуйте готові рішення для маркування семантичних ролей у тексті та гіпотезі (наприклад, [AllenNLP SRL](https://github.com/masrb/allenNLP-SRL) чи [AMR Eager](https://cohort.inf.ed.ac.uk/amreager.html)).

Корисні статті, у яких можна підглянути ознаки:
- [Feature Analysis for Paraphrase Recognition and Textual Entailment](https://pdfs.semanticscholar.org/2d7d/f0b5ac15cdaa50928031f5bb2fc63a0a1f68.pdf), 2013
- [Machine Learning Experiments for Textual Entailment](http://u.cs.biu.ac.il/~nlp/RTE2/Proceedings/02.pdf), 2006
- [A large annotated corpus for learning natural language inference](https://nlp.stanford.edu/pubs/snli_paper.pdf), 2015
- [Learning to recognize features of valid textual entailments](https://nlp.stanford.edu/pubs/rte-naacl06.pdf), 2006
- [Textual entailment](http://www.lsi.upc.edu/~ageno/anlp/textualEntailment.pdf), 2014

Для тренування та тестування використайте **train** та **dev** частини з [The Stanford Natural Language Inference (SNLI) Corpus](https://nlp.stanford.edu/projects/snli/). Протестуйте фінальне рішення на **test**-частині корпусу.

Запишіть ваші спостереження та результати в окремий файл.

## Оцінювання

100% за завдання.

## Крайній термін

09.05.2020


In [98]:
import pandas as pd
import numpy as np

from tqdm import tqdm

In [6]:
train = pd.read_csv(os.path.join(DATA_PATH, "snli_1.0_train.txt"), sep='\t')
dev = pd.read_csv(os.path.join(DATA_PATH, "snli_1.0_dev.txt"), sep='\t')
test = pd.read_csv(os.path.join(DATA_PATH, "snli_1.0_test.txt"), sep='\t')

columns_old = ['sentence1', 'sentence2', 'gold_label']
columns_new = ['premise', 'hypothesis', 'target']
name_mapping = dict(zip(columns_old, columns_new))

train = train.loc[train.gold_label != '-'][columns_old].rename(columns=name_mapping)
dev = dev.loc[dev.gold_label != '-'][columns_old].rename(columns=name_mapping)
test = test.loc[test.gold_label != '-'][columns_old].rename(columns=name_mapping)


print(train.shape, dev.shape, test.shape)

(549367, 3) (9842, 3) (9824, 3)


In [7]:
train.head()

,premise,hypothesis,target
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment


In [8]:
train.target.value_counts(normalize=True)

entailment       0.333868
contradiction    0.333451
neutral          0.332681
Name: target, dtype: float64

In [9]:
dev.target.value_counts(normalize=True)

entailment       0.338244
contradiction    0.333062
neutral          0.328693
Name: target, dtype: float64

In [10]:
test.target.value_counts(normalize=True)

entailment       0.342834
contradiction    0.329499
neutral          0.327667
Name: target, dtype: float64

### Preprocessing

In [47]:
import spacy
import pickle

from multiprocessing import cpu_count
from sklearn.metrics import *

In [12]:
nlp = spacy.load('en_core_web_md')

In [263]:
sample = train.sample()
trgt = sample.target.values[0]
prem = sample.premise.values[0]
hyp = sample.hypothesis.values[0]

In [264]:
print(trgt)
print(prem)
print(hyp)

entailment
A man in a white t-shirt looks toward the camera surrounded by a crowd near a metro station.
The man sees the camera.


In [16]:
def spacy_pipe(df, save_pkl="dev.pkl", batch_size=4096):
    
    prems = nlp.pipe(df["premise"].values, n_threads=4, batch_size=batch_size)
    hyps = nlp.pipe(df["hypothesis"].values, n_threads=4, batch_size=batch_size)
    res = zip(prems, hyps, df["target"].values)
    res = pd.DataFrame(res, columns=columns_new)
    
    if save_pkl:
        res.to_pickle(save_pkl)
    
    return res

In [17]:
print("Pickling:", pd.__version__, spacy.__version__, sep='\n')

Pickling:
0.25.3
2.2.4


In [18]:
%%time

try:
    train = pd.read_pickle("train.pkl")
except FileNotFoundError:
    train = spacy_pipe(train, save_pkl="train.pkl")
    train.to_pickle('train.pkl')

print(train.shape)

(549361, 3)
CPU times: user 3min 18s, sys: 6.31 s, total: 3min 24s
Wall time: 3min 24s


In [29]:
%%time

try:
    dev = pd.read_pickle("dev.pkl")
except FileNotFoundError:
    dev = spacy_pipe(train, save_pkl="dev.pkl")
    dev.to_pickle('dev.pkl')

print(dev.shape)

(9842, 3)
CPU times: user 11.6 s, sys: 436 ms, total: 12.1 s
Wall time: 12.1 s


In [20]:
%%time

try:
    test = pd.read_pickle("test.pkl")
except FileNotFoundError:
    test = spacy_pipe(test)
    test.to_pickle('test.pkl')

print(test.shape)

(9824, 3)
CPU times: user 11.9 s, sys: 405 ms, total: 12.3 s
Wall time: 12.4 s


10k sample for testing hypothesises

In [468]:
# data = train.sample(10000)
# dev_sample = dev.sample(1000)
# test_sample = test.sample(1000)

data = train.sample(100000)
dev_sample = dev
test_sample = test

data.target.value_counts()

neutral          33388
entailment       33312
contradiction    33300
Name: target, dtype: int64

### Simple baseline

In [469]:
def get_features(prem_tkn, hyp_tkn):
    
    features = {
        'prem_tkn_len': len(prem_tkn),
        'hyp_tkn_len': len(hyp_tkn)
    }
    return features

In [470]:
data.groupby('target')['hypothesis', 'premise'].agg([lambda x: x.apply(len).mean()])

,hypothesis,premise
,<lambda>,<lambda>
target,,
contradiction,8.218679,14.162012
entailment,7.466739,14.144993
neutral,9.134659,14.216455


In [471]:
def tkn_num_baseline(prem, hyp):
    prem_len = len(prem)
    hyp_len = len(hyp)
    if prem_len - hyp_len < 4:
        return 'contradiction'
    if prem_len - hyp_len < 5:
        return 'entailment'
    else:
        return 'neutral'

In [472]:
pred = test.apply(lambda x: tkn_num_baseline(x.premise, x.hypothesis), 1)

print(classification_report(test['target'], pred))

               precision    recall  f1-score   support

contradiction       0.33      0.34      0.33      3237
   entailment       0.38      0.08      0.14      3368
      neutral       0.29      0.52      0.37      3219

    micro avg       0.31      0.31      0.31      9824
    macro avg       0.33      0.31      0.28      9824
 weighted avg       0.33      0.31      0.28      9824



### Features

In [473]:
import warnings
warnings.simplefilter("ignore", UserWarning)

from nltk import bigrams
from nltk.corpus import wordnet as wn
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import single_meteor_score

from rouge import Rouge
from jiwer import wer

In [474]:
def syn_overlap(prem_tokens, hyp_tokens):
    pos_map = {'NOUN': wn.NOUN, 'VERB': wn.VERB,
               'ADJ': wn.ADJ, 'ADV': wn.ADV}
    
    common_tokens = token_pos_intersect(prem_tokens, hyp_tokens)
    overlap = 0
    for pair in common_tokens:
        synsets = wn.synsets(pair[0][0], pos = pos_map.get(pair[0][1], None))
        if len(synsets) == 0:
            continue
        synonyms = sum([item.lemma_names()for item in synsets], [])
        if pair[1][0] in synonyms:
            overlap += 1
    return overlap, overlap/len(prem_tokens)


def token_pos_intersect(prem_tokens, hyp_tokens):
    res = []
    for i in prem_tokens:
        for j in hyp_tokens:
            if i[1] == j[1]:
                res.append((i, j))
    return list(set(res))


def token_pos_filter(tokens, pos):
    if pos == 'NOUN':
        return [token[0] for token in tokens if token[1] in ('PROPN', 'NOUN')]
    else:
        return [token[0] for token in tokens if token[1] == pos]

In [475]:
rouge = Rouge()

def get_features(prem, hyp):   
    if isinstance(prem, str):
        prem = nlp(prem)
    if isinstance(hyp, str):
        hyp = nlp(hyp)
        
    prem_lem_pos = [(t.lemma_, t.pos_) for t in prem if not t.is_punct]
    hyp_lem_pos = [(t.lemma_, t.pos_) for t in hyp if not t.is_punct]
    
    prem_lem = [t.lemma_ for t in prem if not t.is_punct]
    hyp_lem = [t.lemma_ for t in hyp if not t.is_punct]
    bi_prem_lem = list(bigrams(prem_lem))
    bi_hyp_lem = list(bigrams(hyp_lem))
    
    prem_lemma_txt = " ".join(prem_lem)
    hyp_lemma_txt = " ".join(hyp_lem)
    
    lemma_overlap = list(set(prem_lem).intersection(set(hyp_lem)))
    lemma_overlap_count = len(lemma_overlap)
    lemma_overlap_ratio = lemma_overlap_count/len(set(prem_lem))
    bigram_overlap = list(set(list(bi_prem_lem)).intersection(set(list(bi_hyp_lem))))
    
    syn_overlap_count, syn_overlap_ratio = syn_overlap(prem_lem_pos, hyp_lem_pos)
            
    prem_nouns = set(token_pos_filter(prem_lem_pos, 'NOUN'))
    noun_overlap = prem_nouns.intersection(set(token_pos_filter(hyp_lem_pos, 'NOUN')))
    noun_overlap_count = len(noun_overlap)
    noun_overlap_ratio = noun_overlap_count / len(prem_nouns) if len(prem_nouns) > 0 else 0
    
    prem_verbs = set(token_pos_filter(prem_lem_pos, 'VERB'))
    verb_overlap = prem_verbs.intersection(set(token_pos_filter(hyp_lem_pos, 'VERB')))
    verb_overlap_count = len(verb_overlap)
    verb_overlap_ratio = verb_overlap_count / len(prem_verbs) if len(prem_verbs) > 0 else 0
    
    prem_adjs = set(token_pos_filter(prem_lem_pos, 'ADJ'))
    adj_overlap = prem_adjs.intersection(set(token_pos_filter(hyp_lem_pos, 'ADJ')))
    adj_overlap_count = len(adj_overlap)
    adj_overlap_ratio = adj_overlap_count / len(prem_adjs) if len(prem_adjs) > 0 else 0
    
    prem_advs = set(token_pos_filter(prem_lem_pos, 'ADV'))
    adv_overlap = prem_advs.intersection(set(token_pos_filter(hyp_lem_pos, 'ADV')))
    adv_overlap_count = len(adv_overlap)
    adv_overlap_ratio = adv_overlap_count / len(prem_advs) if len(prem_advs) > 0 else 0
    
    rouge_scores = rouge.get_scores(hyps=hyp_lemma_txt, refs=prem_lemma_txt)[0]

    features = {
        # base features
        'prem_lem_len': len(prem_lem),
        'hyp_lem_len': len(hyp_lem),
        # overlap features
        'lemma_overlap_count': lemma_overlap_count,
        'lemma_overlap_ratio': lemma_overlap_ratio,
        'noun_overlap_count': noun_overlap_count,
        'noun_overlap_ratio': noun_overlap_ratio,
        'verb_overlap_count': verb_overlap_count,
        'verb_overlap_ratio': verb_overlap_ratio,
        'adj_overlap_count': adj_overlap_count,
        'adj_overlap_ratio': adj_overlap_ratio,
        'adv_overlap_count': adv_overlap_count,
        'adv_overlap_ratio': adv_overlap_ratio,
        # WordNet synonyms intersection
        'syn_overlap_count': syn_overlap_count,
        'syn_overlap_ratio': syn_overlap_ratio,
        # BLEU
        'bleu1': sentence_bleu([prem_lem], hyp_lem, weights=(1, 0, 0, 0)),
        'bleu2': sentence_bleu([prem_lem], hyp_lem, weights=(0, 1, 0, 0)),
        'bleu3': sentence_bleu([prem_lem], hyp_lem, weights=(0, 0, 1, 0)),
        'bleu4': sentence_bleu([prem_lem], hyp_lem, weights=(0, 0, 0, 1)),
        'bleu_cum': sentence_bleu([prem_lem], hyp_lem, weights=(0.25, 0.25, 0.25, 0.25)),
        # Rouge
        "rouge-1-f": rouge_scores.get('rouge-1', {}).get('f', 0),
        "rouge-2-f": rouge_scores.get('rouge-2', {}).get('f', 0),
        "rouge-l-f": rouge_scores.get('rouge-l', {}).get('f', 0),
        # Word accuracy
        "wacc": 1 - wer(prem_lemma_txt, hyp_lemma_txt),
        # METEOR
        "meteor": single_meteor_score(reference=prem_lemma_txt, hypothesis=hyp_lemma_txt),
    }
    # unigrams and bigrams for premise
    for gram in prem_lem:
        features[f'prem_{gram}'] = features.get(f'prem_{gram}', 0) + 1
    for gram in bi_prem_lem:
        features[f'prem_{gram[0]}_{gram[1]}'] = features.get(f'prem_{gram[0]}_{gram[1]}', 0) + 1
    # unigrams and bigrams for hypothesis
    for gram in hyp_lem:
        features[f'hyp_{gram}'] = features.get(f'hyp_{gram}', 0) + 1
    for gram in bi_hyp_lem:
        features[f'hyp_{gram[0]}_{gram[1]}'] = features.get(f'hyp_{gram[0]}_{gram[1]}', 0) + 1
    # unigrams and bigrams for overlaping
    for gram in lemma_overlap:
        features[f'common_{gram}'] = features.get(f'common_{gram}', 0) + 1
    for gram in bigram_overlap:
        features[f'common_{gram[0]}_{gram[1]}'] = features.get(f'common_{gram[0]}_{gram[1]}', 0) + 1

    return features

In [476]:
print(prem)
print(hyp)

get_features(prem, hyp)

A man in a white t-shirt looks toward the camera surrounded by a crowd near a metro station.
The man sees the camera.


{'prem_lem_len': 19,
 'hyp_lem_len': 5,
 'lemma_overlap_count': 3,
 'lemma_overlap_ratio': 0.1875,
 'noun_overlap_count': 2,
 'noun_overlap_ratio': 0.2857142857142857,
 'verb_overlap_count': 0,
 'verb_overlap_ratio': 0.0,
 'adj_overlap_count': 0,
 'adj_overlap_ratio': 0.0,
 'adv_overlap_count': 0,
 'adv_overlap_ratio': 0,
 'syn_overlap_count': 3,
 'syn_overlap_ratio': 0.15789473684210525,
 'bleu1': 0.03648603757513078,
 'bleu2': 0.015202515656304493,
 'bleu3': 1.35306880681562e-309,
 'bleu4': 1.35306880681562e-309,
 'bleu_cum': 5.645084941608842e-156,
 'rouge-1-f': 0.24999999670138892,
 'rouge-2-f': 0.09090908793388439,
 'rouge-l-f': 0.2999999968,
 'wacc': 0.1578947368421053,
 'meteor': 0.17933238636363635,
 'prem_a': 4,
 'prem_man': 1,
 'prem_in': 1,
 'prem_white': 1,
 'prem_t': 1,
 'prem_shirt': 1,
 'prem_look': 1,
 'prem_toward': 1,
 'prem_the': 1,
 'prem_camera': 1,
 'prem_surround': 1,
 'prem_by': 1,
 'prem_crowd': 1,
 'prem_near': 1,
 'prem_metro': 1,
 'prem_station': 1,
 'prem_a

In [477]:
train_features, train_labels = [], []

for i, row in tqdm(data.iterrows()):
    train_features.append(get_features(row['premise'], row['hypothesis']))
    train_labels.append(row['target'])

100000it [03:14, 513.01it/s]


In [478]:
dev_features, dev_labels = [], []

for i, row in tqdm(dev_sample.iterrows()):
    dev_features.append(get_features(row['premise'], row['hypothesis']))
    dev_labels.append(row['target'])

9842it [00:19, 493.97it/s]


In [479]:
test_features, test_labels = [], []

for i, row in tqdm(test_sample.iterrows()):
    test_features.append(get_features(row['premise'], row['hypothesis']))
    test_labels.append(row['target'])

9824it [00:19, 502.21it/s]


### Modeling

In [480]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [481]:
RANDOM_STATE = 0

In [482]:
vec = DictVectorizer()

In [483]:
%%time

train_vec = vec.fit_transform(train_features)
dev_vec = vec.transform(dev_features)
test_vec = vec.transform(test_features)

print(f"Number of features: {len(vec.vocabulary_)}")

Number of features: 266478
CPU times: user 4.51 s, sys: 67.9 ms, total: 4.58 s
Wall time: 4.58 s


#### Logistic regression

In [484]:
lr_clf = LogisticRegression(penalty='l1', random_state=RANDOM_STATE)

lr_clf.fit(train_vec, train_labels)

/home/dima/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/dima/anaconda3/envs/p36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [485]:
pred = lr_clf.predict(dev_vec)

print(classification_report(pred, dev_labels))

               precision    recall  f1-score   support

contradiction       0.74      0.73      0.74      3315
   entailment       0.81      0.76      0.78      3559
      neutral       0.65      0.71      0.68      2968

    micro avg       0.73      0.73      0.73      9842
    macro avg       0.73      0.73      0.73      9842
 weighted avg       0.74      0.73      0.74      9842



In [486]:
pred = lr_clf.predict(test_vec)

print(classification_report(pred, test_labels))

               precision    recall  f1-score   support

contradiction       0.73      0.73      0.73      3227
   entailment       0.79      0.75      0.77      3550
      neutral       0.66      0.69      0.67      3047

    micro avg       0.73      0.73      0.73      9824
    macro avg       0.73      0.73      0.73      9824
 weighted avg       0.73      0.73      0.73      9824



In [504]:
import eli5
eli5.show_weights(lr_clf, vec=vec)

#### LigthGBM

In [487]:
from lightgbm import LGBMClassifier
from sklearn.decomposition import TruncatedSVD

In [490]:
N_COMP = 600

In [491]:
def lgb_fscore(y_true, y_pred):
    y_pred = y_pred.reshape(len(np.unique(y_true)), -1)
    y_pred = y_pred.argmax(axis=0)
    res = f1_score(y_true, y_pred, average='macro')
    return 'macro_f1', res, True

In [492]:
%%time

t_svd = TruncatedSVD(n_components=N_COMP)

train_trunc = t_svd.fit_transform(train_vec)
dev_trunc = t_svd.transform(dev_vec)
test_trunc = t_svd.transform(test_vec)

CPU times: user 3min 52s, sys: 12.4 s, total: 4min 4s
Wall time: 1min 47s


In [497]:
params = {
    'num_class': 3,
    'num_rounds': 10000,
    'max_depth': -1,
    'learning_rate': 0.01,
    'num_leaves': 20,
    'verbose': 100,
    'early_stopping_rounds': 300,
#     'min_data_in_leaf': 30,
#     'lambda_l2': 0.7,
#     'feature_fraction': 0.2,
    'metric': 'custom',
    'random_state': RANDOM_STATE
}

lgb_clf = LGBMClassifier(**params)

In [498]:
lgb_clf.fit(
    X=train_trunc,
    y=train_labels,
    eval_set=[(dev_trunc, dev_labels)],
    verbose=params['verbose'],
    eval_metric=lgb_fscore,
)

Training until validation scores don't improve for 300 rounds
[100]	valid_0's macro_f1: 0.538726
[200]	valid_0's macro_f1: 0.549392
[300]	valid_0's macro_f1: 0.560046
[400]	valid_0's macro_f1: 0.569111
[500]	valid_0's macro_f1: 0.579588
[600]	valid_0's macro_f1: 0.587769
[700]	valid_0's macro_f1: 0.592413
[800]	valid_0's macro_f1: 0.597854
[900]	valid_0's macro_f1: 0.602666
[1000]	valid_0's macro_f1: 0.608244
[1100]	valid_0's macro_f1: 0.609291
[1200]	valid_0's macro_f1: 0.612713
[1300]	valid_0's macro_f1: 0.614697
[1400]	valid_0's macro_f1: 0.616455
[1500]	valid_0's macro_f1: 0.617898
[1600]	valid_0's macro_f1: 0.621129
[1700]	valid_0's macro_f1: 0.623315
[1800]	valid_0's macro_f1: 0.624122
[1900]	valid_0's macro_f1: 0.62586
[2000]	valid_0's macro_f1: 0.62732
[2100]	valid_0's macro_f1: 0.627884
[2200]	valid_0's macro_f1: 0.628082
[2300]	valid_0's macro_f1: 0.631329
[2400]	valid_0's macro_f1: 0.63163
[2500]	valid_0's macro_f1: 0.632587
[2600]	valid_0's macro_f1: 0.633297
[2700]	valid_0

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        early_stopping_rounds=300, importance_type='split',
        learning_rate=0.01, max_depth=-1, metric='custom',
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=100, n_jobs=-1, num_class=3, num_leaves=20,
        num_rounds=10000, objective=None, random_state=0, reg_alpha=0.0,
        reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0, verbose=100)

In [501]:
pred = lgb_clf.predict(dev_trunc)

print(classification_report(pred, dev_labels))

               precision    recall  f1-score   support

contradiction       0.63      0.63      0.63      3258
   entailment       0.73      0.69      0.71      3502
      neutral       0.58      0.61      0.59      3082

    micro avg       0.65      0.65      0.65      9842
    macro avg       0.64      0.64      0.64      9842
 weighted avg       0.65      0.65      0.65      9842



In [502]:
pred = lgb_clf.predict(test_trunc)

print(classification_report(pred, test_labels))

               precision    recall  f1-score   support

contradiction       0.64      0.64      0.64      3215
   entailment       0.73      0.69      0.71      3538
      neutral       0.58      0.61      0.59      3071

    micro avg       0.65      0.65      0.65      9824
    macro avg       0.65      0.65      0.65      9824
 weighted avg       0.65      0.65      0.65      9824

